# Business Case: Modelagem de Swaps para um Cabinet

Para a modelagem da série temporal de swaps de um cabinet em específico, decidi escolher o **cabinet de id 13**, que possui uma boa quantidade de swaps comparado aos outros. Assim, o estudo e previsões a seguir serão baseados neste cabinet, servindo como caso ilustrativo para o restante da case.

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

from sktime.performance_metrics.forecasting import mean_absolute_percentage_error, mean_absolute_error
from statsmodels.discrete.count_model import ZeroInflatedPoisson, ZeroInflatedNegativeBinomialP

import plotly.express as px

In [ ]:
df = pd.read_csv('cabinet_data.csv')

## Gráficos Descritivos

Vamos começar analisando alguns gráficos para melhor entender o comportamendo da nossa variável alvo.

In [ ]:
fig = px.histogram(df, 'counts')

fig.update_layout(
    bargap=0.5,
    xaxis_dtick=1,
    title='Histograma de Swaps Horários',
    yaxis_title='Contagem',
    xaxis_title='Swaps',
    height=500, width=1000
)

fig.show()

In [ ]:
fig = px.line(df, x='hour', y='counts')

fig.update_layout(
    title='Série Horária de Swaps',
    yaxis_title='Swaps',
    xaxis_title='Data/hora',
    height=500, width=1000
)

fig.show()

In [ ]:
mean_counts = df['counts'].mean()
var_counts = df['counts'].var()

print(f"Média: {mean_counts:.2f}")
print(f"Variância: {var_counts:.2f}")
print(f"Razão variância/média: {var_counts/mean_counts:.2f}")

Como pode-se observar nos gráficos acima, os valores da série temporal são baixos e com uma presença significativa de zeros. A princípio, podemos pensar em um modelo Poisson para descrever esse tipo de distribuição. Mas as estatísticas descritivas da série indicam uma média de 2.09 e uma variância de 3.81, resultando em uma razão variância/média de 1.82. Esse resultado sugere que a série apresenta **sobredispersão**, ou seja, a variabilidade dos dados é maior do que a esperada para uma distribuição Poisson padrão.

Diante dessas características, foram testados modelos adequados para contagem de eventos com baixa frequência e excesso de zeros:

- Poisson: modelo clássico para contagem de eventos, assume que a média e a variância são iguais.

- Negative Binomial: lida com overdispersion, permitindo que a variância seja maior que a média.

- Zero-Inflated Poisson (ZIP): combina um modelo Poisson com um componente que modela explicitamente o excesso de zeros.

Mas antes de escrever os modelos, vamos criar algumas features temporais que possivelmente auxiliaram na hora de explicar o comportamento da variável.

## Modelagem

In [ ]:
df['hour'] = pd.to_datetime(df['hour'])

# Hora e dia da semana
df['hour_of_day'] = df['hour'].dt.hour
df['day_of_week'] = df['hour'].dt.dayofweek 
df['is_weekend']  = (df['day_of_week'] >= 5).astype(int)

# Lags
for lag in [1, 2, 3, 24, 48]:
    df[f'lag_{lag}'] = df['counts'].shift(lag).fillna(0)

df['day_of_week']  = df['day_of_week'].astype('category')

# Média móvel
df['rolling_mean_3h'] = df['counts'].shift(1).rolling(window=3).mean().fillna(df['counts'].mean())
df['rolling_mean_24h'] = df['counts'].shift(1).rolling(window=24).mean().fillna(df['counts'].mean())

Agora que temos as features temporais criadas, vamos testar os modelos.

In [ ]:
# Fórmula do modelo
formula = 'counts ~ C(hour_of_day) + C(day_of_week) + is_weekend + lag_1 + lag_2 + lag_3 + lag_24 + lag_48 + rolling_mean_3h'

# --- Ajustes ---
# Poisson
poisson_model = sm.GLM.from_formula(formula, data=df, family=sm.families.Poisson()).fit()

# NB
nb_model = sm.GLM.from_formula(formula, data=df, family=sm.families.NegativeBinomial()).fit()

# ZIP
zip_model = ZeroInflatedPoisson.from_formula(formula, data=df, inflation='logit').fit()

# Previsões
df['predicted_poisson'] = poisson_model.predict(df).round()
df['predicted_nb'] = nb_model.predict(df).round()
df['predicted_zip'] = zip_model.predict(df).round()

Para decidirmos com qual modelo seguir, vamos utilizar a métrica do erro absoluto médio (MAE), pois ela faz mais sentido para séries inteiras e com muito zeros. Sendo assim, o modelo com o menor erro será escolhido.

In [ ]:
maes = {}
for model in ['predicted_poisson', 'predicted_nb', 'predicted_zip']: 
    maes[model] = mean_absolute_error(df['counts'], df[model]).round(4)
maes

Apesar das diferenças teóricas, a avaliação de desempenho mostrou que o modelo Poisson apresentou os melhores resultados preditivos (por mais que minimamente). Isso indica que, embora houvesse algum excesso de zeros e leve overdispersion, o comportamento da série pôde ser capturado de forma mais simples e robusta pelo modelo Poisson, tornando-o a escolha mais adequada para a previsão da demanda horária de swaps neste cabinet em específico. Agora, vamos avaliar a sua taxa de acertos de zeros.

In [ ]:
zero_hits = ((df['counts'] == 0) & (df['predicted_poisson'] == 0)).sum()
total_zeros = (df['counts'] == 0).sum()

taxa_acerto_zeros = zero_hits / total_zeros
print(f"Taxa de acerto de zeros: {round(taxa_acerto_zeros*100, 2)}%")

Após ter selecionado nosso modelo, vamos gerar previsões horárias para uma semana a frente considerando um aumento de 10%, e finalmente analisar o gráfico de comparação entre o real e o previsto.

In [ ]:
# Dataframe futuro
last_datetime = df['hour'].max()
future_dates = pd.date_range(start=last_datetime + pd.Timedelta(hours=1), periods=24*7, freq='H')
df_future = pd.DataFrame({'hour': future_dates})

# Features temporais
df_future['hour_of_day'] = df_future['hour'].dt.hour
df_future['day_of_week'] = df_future['hour'].dt.dayofweek
df_future['is_weekend'] = (df_future['hour'].dt.dayofweek >= 5).astype(int)

# Inicializar lags (com últimos valores reais)
last_counts = df['counts'].values.tolist()
preds = []

for t in range(len(df_future)):
    lag_1 = last_counts[-1] if t < 1 else pred_nb[t-1]
    lag_2 = last_counts[-2] if t < 2 else pred_nb[t-2]
    lag_3 = last_counts[-3] if t < 3 else pred_nb[t-3]
    lag_24 = last_counts[-24] if t < 24 else pred_nb[t-24]
    lag_48 = last_counts[-48] if t < 48 else pred_nb[t-48]

    if t < 3:
        rolling_mean_3h = np.mean(last_counts[-3+t:] + pred_nb[:t])
    else:
        rolling_mean_3h = np.mean(preds[t-3:t])

    X_t = pd.DataFrame({
        'hour_of_day': [df_future.loc[t, 'hour_of_day']],
        'day_of_week': [df_future.loc[t, 'day_of_week']],
        'is_weekend': [df_future.loc[t, 'is_weekend']],
        'lag_1': [lag_1],
        'lag_2': [lag_2],
        'lag_3': [lag_3],
        'lag_24': [lag_24],
        'lag_48': [lag_48],
        'rolling_mean_3h': [rolling_mean_3h]
    })

    pred_t = poisson_model.predict(X_t)[0]
    preds.append(pred_t)

# Adicionar previsões ao DataFrame futuro
df_future['predicted_poisson'] = np.round(np.array(preds)*1.1)

In [ ]:
df_final = pd.concat([df, df_future])

In [ ]:
fig = px.line(df_final, x='hour', y=['counts','predicted_poisson'])

fig.update_layout(
    title='Comparação entre o Real e o Previsto',
    yaxis_title='Swaps',
    xaxis_title='Data/hora',
    height=500, width=1000
)

fig.show()

Para incorporar o crescimento de 10% nos dados, optei por aplicar o ajuste diretamente às previsões do modelo, e não aos dados históricos. Essa abordagem mantém a integridade do padrão aprendido pela série temporal, e tenta refletir de forma mais realista o impacto do aumento nas trocas futuras.

## Conclusão

A análise mostrou que o modelo escolhido se ajustou de forma satisfatória à série temporal de swaps do cabinet de ID 13, conseguindo capturar tendências temporais relevantes da demanda horária. O desempenho do modelo foi consistente, apresentando um **MAE inferior a 1**, o que indica que, em média, o erro nas previsões é inferior a um swap por hora — um resultado relevante considerando os valores baixos da série.

No entanto, ao estender a previsão para uma semana à frente, observa-se uma dificuldade do modelo em replicar picos e vales, resultando em um leve desfasamento das previsões. Isso sugere que, embora o modelo seja eficiente para curto prazo, sua acurácia diminui para projeções mais longas, possivelmente devido à variabilidade intrínseca da série.

Em suma, o modelo oferece boas previsões horárias de swaps em curto prazo, sendo útil para planejamento operacional, mas requer atenção ao extrapolar previsões para horizontes mais longos.